# New Code

In [ ]:
import requests
import json
import bs4
import pandas as pd
import os
import urllib.request
import re
from bs4 import BeautifulSoup

!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

## 2005-2007 (1-6)

In [ ]:
wd.get('https://web.archive.org/web/20071127152318/http://www.cmfu.com/topten.asp')
soup = BeautifulSoup(wd.page_source, 'html.parser')

link = [] ; image=[]; title = [] ; author = []; category=[]

for i in soup.find_all(name='tr',attrs={'height':'25'}):
  title.append(i.find_next('a').text)
  author.append(i.find_next('a').find_next(name='td',attrs={'align':'center'}).text)
  category.append(i.find_next('td').find_next('td').text)
  book_id = i.find_next('a')['href'].partition('=')[-1]
  image.append(f'https://web.archive.org/web/20071127152318im_/http://newauthor7.cmfu.com/books/{book_id}/{book_id}.jpg')
  link.append(f'https://web.archive.org/web/20071127152318/http://www.cmfu.com/showbook.asp?Bl_id={book_id}')

# make a dataframe from all metadata
book_id = list(range(0, len(link)))
d = {'Id':book_id,'Title': title, 'Author':author,'Category':category,'Image':image,'Link':link}
df = pd.DataFrame(d)

# save the dataframe
df.to_excel('book_table_6.xlsx')

# create the image folder
os.mkdir('/content/images_6')

# download images
for i in range(len(link)):
  try:
    urllib.request.urlretrieve(image[i], f"/content/images_6/book{i}.jpg")
  except:
    try:
      urllib.request.urlretrieve(image[i].replace('7.c','6.c'), f"/content/images_6/book{i}.jpg")
    except:
      try:
        urllib.request.urlretrieve(image[i].replace('7.c','8.c'), f"/content/images_6/book{i}.jpg")
      except:
        try:
          urllib.request.urlretrieve(image[i].replace('7.c','2.c'), f"/content/images_6/book{i}.jpg")
        except:
          try:
            urllib.request.urlretrieve(image[i].replace('7.c','5.c'), f"/content/images_6/book{i}.jpg")
          except:
            try:
              urllib.request.urlretrieve(image[i].replace('7.c','3.c'), f"/content/images_6/book{i}.jpg")
            except:
              pass

# zip all files in the folder
!zip -r /content/images_6.zip /content/images_6

# download the zip file
from google.colab import files
files.download('images_6.zip')
files.download('book_table_6.xlsx')

## 2008-2009 (7-10)

In [ ]:
wd.get('https://web.archive.org/web/20080421093642/http://www.qidian.com/Book/BookList.aspx')
soup = BeautifulSoup(wd.page_source, 'html.parser')

link = [] ; image=[]; title = [] ; author = []; category=[]

for i in soup.find_all(name='a',attrs={'class':'tsn'}):
  title.append(i.text)
  category.append(i['title'])
  try:
    book_id = re.search('http://www.qidian.com/Book/(.*).aspx', i['href']).group(1)
    image.append(f'https://web.archive.org/web/20080421093642im_/http://image.cmfu.com/books/{book_id}/{book_id}.jpg')
    link.append(f'https://web.archive.org/web/20080421093642mp_/http://www.qidian.com/Book/{book_id}.aspx')
  except:
    image.append('NaN')
    link.append('NaN')

# make a dataframe from all metadata
book_id = list(range(0, len(link)))
d = {'Id':book_id,'Title': title, 'Category':category,'Image':image,'Link':link}
df = pd.DataFrame(d)

# save the dataframe
df.to_excel('book_table_7.xlsx')

# create the image folder
os.mkdir('/content/images_7')

# download images
for i in range(len(link)):
  try:
    urllib.request.urlretrieve(image[i], f"/content/images_7/book{i}.jpg")
  except:
    pass

# zip all files in the folder
!zip -r /content/images_7.zip /content/images_7

# download the zip file
from google.colab import files
files.download('images_7.zip')
files.download('book_table_7.xlsx')

## 2010-2016

In [ ]:
for i in soup.find_all(name='b',attrs={'class':'lei'}):
  print(i.find_next('a').find_next('a')['href'])

In [ ]:
wd.get('https://web.archive.org/web/20170217233938/http://top.qdmm.com/MMWeb/TopDetail.aspx?TopType=7&Time=1')
soup = BeautifulSoup(wd.page_source, 'html.parser')

link = [] ; image=[]; title = [] ; author = []; category=[]

for i in soup.find_all(name='b',attrs={'class':'lei'}):
  category.append(i.text)
  title.append(i.find_next('a').find_next('a')['title'])
  author.append(i.find_next(name='b',attrs={'class':'author'}).text)
  try:
    book_id = re.search('http://www.qdmm.com/MMWeb/(.*).aspx', i.find_next('a').find_next('a')['href']).group(1)
    image.append(f'https://web.archive.org/web/20170217233938im_/http://image.cmfu.com/books/{book_id}/{book_id}.jpg')
    link.append(f'https://web.archive.org/web/20170217233938/http://www.qdmm.com/mmweb/{book_id}.aspx')
  except:
    image.append('NaN')
    link.append('NaN')

# make a dataframe from all metadata
book_id = list(range(0, len(link)))
d = {'Title': title, 'Author':author,'Category':category,'Image':image,'Link':link}
df_2017 = pd.DataFrame(d)
df_2017

In [ ]:
# merge all dataframes
df_final = pd.DataFrame()
df_list = [df_2015, df_2016, df_2017]

for i in df_list:
  df_final=df_final.append(i)

df_final = df_final.reset_index(drop=True)

# save the dataframe
df_final.to_excel('girl_monthly_2015_2020.xlsx')

In [ ]:
# create the image folder
os.mkdir('/content/girl_monthly_2015_2020_images')

# download images
for i in range(len(df_final)):
  try:
    urllib.request.urlretrieve(df_final['Image'][i], f"/content/girl_monthly_2015_2020_images/book{i}.jpg")
  except:
    pass

# zip all files in the folder
!zip -r /content/girl_monthly_2015_2020_images.zip /content/girl_monthly_2015_2020_images

# download the zip file
from google.colab import files
files.download('girl_monthly_2015_2020_images.zip')
files.download('girl_monthly_2015_2020.xlsx')

### girl site

In [ ]:
# create individual dataframes
wd.get('https://web.archive.org/web/20151022235243/http://www.qdmm.com/MMWeb/TopDetail.aspx?TopType=7&Time=1')
soup = BeautifulSoup(wd.page_source, 'html.parser')

link = [] ; image=[]; title = [] ; author = []; category=[]

for i in soup.find_all(name='td',attrs={'width':'18%'}):
  title.append(i.text)
  author.append(i.find_next(name='td',attrs={'width':'15%'}).text)
  category.append(i.previous_sibling.text)
  book_id = book_id = re.search('http://www.qdmm.com/MMWeb/(.*).aspx', i.find_next('a')['href']).group(1)
  image.append(f'https://web.archive.org/web/20140303110544im_/http://image.cmfu.com/books/{book_id}/{book_id}.jpg')
  link.append(f'https://web.archive.org/web/20140303110544/http://www.qdmm.com/mmweb/{book_id}.aspx')

# make a dataframe from all metadata
book_id = list(range(0, len(link)))
d = {'Title': title, 'Author':author,'Category':category,'Image':image,'Link':link}
df_2014a = pd.DataFrame(d)
df_2014a

In [ ]:
# merge all dataframes
df_final = pd.DataFrame()
df_list = [df_2011a, df_2011b, df_2012a, df_2012b, df_2013a, df_2013b,df_2014a]

for i in df_list:
  df_final=df_final.append(i)

df_final = df_final.reset_index(drop=True)

# save the dataframe
df_final.to_excel('girl_monthly_2010_2014.xlsx')

In [ ]:
# create the image folder
os.mkdir('/content/girl_monthly_2010_2014_images')

# download images
for i in range(len(df_final)):
  try:
    urllib.request.urlretrieve(df_final['Image'][i], f"/content/girl_monthly_2010_2014_images/book{i}.jpg")
  except:
    pass

# zip all files in the folder
!zip -r /content/girl_monthly_2010_2014_images.zip /content/girl_monthly_2010_2014_images

# download the zip file
from google.colab import files
files.download('girl_monthly_2010_2014_images.zip')
files.download('girl_monthly_2010_2014.xlsx')

### main site

In [ ]:
# create individual dataframes
wd.get('https://web.archive.org/web/20110122124718/http://www.qdmm.com/MMWeb/MMBookSortList.aspx?Type=MV')
soup = BeautifulSoup(wd.page_source, 'html.parser')

link = [] ; image=[]; title = [] ; author = []; category=[]

for i in soup.find(name='tbody').find_all(name='a',attrs={'class':'type'}):
  category.append(i.text)

for i in soup.find(name='tbody').find_all(name='a',attrs={'class':'name'}):
  title.append(i.text)
  book_id = re.search('http://www.qidian.com/Book/(.*).aspx', i['href']).group(1)
  image.append(f'http://web.archive.org/web/20110122124718im_/http://image.cmfu.com/books/{book_id}/{book_id}.jpg')
  link.append(f'https://web.archive.org/web/20110122124718/http://www.qidian.com/Book/{book_id}.aspx')

for i in soup.find(name='tbody').find_all(name='a',attrs={'class':'author'}):
  author.append(i.text)

# make a dataframe from all metadata
book_id = list(range(0, len(link)))
d = {'Title': title, 'Author':author,'Category':category,'Image':image,'Link':link}
df_2011a = pd.DataFrame(d)
df_2011a

,Title,Author,Category,Image,Link


In [ ]:
# merge all dataframes
df_final = pd.DataFrame()
df_list = [df_2015, df_2016]

for i in df_list:
  df_final=df_final.append(i)

df_final = df_final.reset_index(drop=True)

# save the dataframe
df_final.to_excel('monthly_pk_2015_2016.xlsx')

In [ ]:
# create the image folder
os.mkdir('/content/monthly_pk_2015_2016_images')

# download images
for i in range(len(df_final)):
  try:
    urllib.request.urlretrieve(df_final['Image'][i], f"/content/monthly_pk_2015_2016_images/book{i}.jpg")
  except:
    pass

# zip all files in the folder
!zip -r /content/monthly_pk_2015_2016_images.zip /content/monthly_pk_2015_2016_images

# download the zip file
from google.colab import files
files.download('monthly_pk_2015_2016_images.zip')
files.download('monthly_pk_2015_2016.xlsx')

## 2017-2020

In [ ]:
wd.get('https://web.archive.org/web/20200829084158/https://www.qidian.com/rank/signnewbook')
soup = BeautifulSoup(wd.page_source, 'html.parser')

link = [] ; image=[]; title = [] ; author = []; category=[]

for i in soup.find_all(name='div',attrs={'class':'book-mid-info'}):
  title.append(i.find('h4').text)
  author.append(i.find(name='p',attrs={'class':'author'}).text.partition('|')[0])
  category.append(i.find(name='p',attrs={'class':'author'}).text.partition('|')[-1])
  image.append(i.previous_sibling.previous_sibling.find('img')['src'])
  link.append(i.previous_sibling.previous_sibling.find('a')['href'])

# make a dataframe from all metadata
book_id = list(range(0, len(link)))
d = {'Title': title, 'Author':author,'Category':category,'Image':image,'Link':link}
df_2020b = pd.DataFrame(d)
df_2020b

In [ ]:
# merge all dataframes
df_final = pd.DataFrame()
df_list = [df_2017,df_2018a,df_2018b,df_2019a,df_2019b,df_2020a,df_2020b]

for i in df_list:
  df_final=df_final.append(i)

df_final = df_final.reset_index(drop=True)

# save the dataframe
df_final.to_excel('new_author_2017_2020.xlsx')

In [ ]:
# create the image folder
os.mkdir('/content/new_author_2017_2020_images')

# download images
for i in range(len(df_final)):
  try:
    urllib.request.urlretrieve(df_final['Image'][i], f"/content/new_author_2017_2020_images/book{i}.jpg")
  except:
    pass

# zip all files in the folder
!zip -r /content/new_author_2017_2020_images.zip /content/new_author_2017_2020_images

# download the zip file
from google.colab import files
files.download('new_author_2017_2020_images.zip')
files.download('new_author_2017_2020.xlsx')

# Old Code

## Javascript Coding

In [ ]:
# CMFU 2005-2007 所有榜单
cmfu= ['https://web.archive.org/web/20050301030212/http://cmfu.com/topten.asp',
'https://web.archive.org/web/20051101011254/http://cmfu.com/topten.asp',
'https://web.archive.org/web/20060430201710/http://cmfu.com/topten.asp',
'https://web.archive.org/web/20061205090106/http://cmfu.com/topten.asp',
'https://web.archive.org/web/20070406203456/http://cmfu.com/topten.asp',
'https://web.archive.org/web/20071127152318/http://cmfu.com/topten.asp']

# retrive javascript file to putlist with the web item id
for i in range(len(cmfu)):
  cmfu[i]='https://web.archive.org/web/'+cmfu[i][-41:-27]+'js_/http://cmfu.com/script/list_top3.js'

In [ ]:
# PARSE THE JAVASCRIPT IN THE CMFU LIST
URL = cmfu[0]
page = requests.get(URL)
soup = page.text

# use regex to retrieve substrings
import re
metadata_ls = re.findall('\(.*?\)',soup)
del metadata_ls[0:12]
del metadata_ls[-7:]

# create a dataframe with all relevant metadata
df = pd.DataFrame(metadata_ls)
df = df[0].str.replace("'",'')
df = df.to_frame()
df = df[0].str.split(',',expand=True)

df_new = pd.DataFrame()
df_new[['book_id','title','author','category']]  = df[[1,2,3,5]]

df_new['image'] = 'https://web.archive.org/web/20050305213648im_/http://author.cmfu.com/bookimgs/' + df_new['book_id'] + '.jpg'
df_new

In [ ]:
# SAVE THE NEW DATAFRAME
df_new.to_excel('book_table_2.xlsx')

# create the image folder
os.mkdir('/content/images_2')

# download images
for i in range(len(df_new)):
  try:
    urllib.request.urlretrieve(df_new['image'][i], f"/content/images_2/book{i}.jpg")
  except:
    pass

# zip all files in the folder
!zip -r /content/images_2.zip /content/images_2

# download the zip file
from google.colab import files
files.download('images_1.zip')
files.download('book_table_2.xlsx')

## Qidian 2008 - 2016 Part 1

In [ ]:
# PARSE THE JAVASCRIPT IN THE CMFU LIST
URL = 'https://web.archive.org/web/20080421093642js_/http://script.cmfu.com/script/BoardScript.js'
page = requests.get(URL)
soup = page.text

# use regex to retrieve substrings
import re
metadata_ls = re.findall('\(.*?\)',soup)
del metadata_ls[0:13]
del metadata_ls[-6:]

# create a dataframe with all relevant metadata
df = pd.DataFrame(metadata_ls)
df = df[0].str.replace("'",'')
df = df.to_frame()
df = df[0].str.split(',',expand=True)
df

,0,1,2,3,4,5,6,7,8,9
0,(1797814,召唤悍妞,2217003,共工天水,685213,31180029,卷二 八十八、自曝,8,东方玄幻),None
1,(1775841,异世药王,2108391,独悠,655724,31180884,第五卷 灵御城！ 第一八章 药师会堂,73,异界大陆),None
2,(1209977,斗破苍穹,1019021,天蚕土豆,563468,29796715,作品相关 薰儿的图片,73,异界大陆),None
3,(1264634,仙逆,1357513,耳根,416025,28536969,作品相关 三江访谈 感言,44,奇幻修真),None
4,(1789892,最轮回,2228622,王袍,366735,31179707,正文 第112章 事实胜于雄辩,12,都市生活),None
...,...,...,...,...,...,...,...,...,...,...
902,(1515258,人生若只待初见,0,,661,0,,0,),None
903,(1209977,斗破苍穹,0,,635,0,,0,),None
904,(1601862,闪魂,0,,631,0,,0,),None
905,(1454337,九转无天,0,,553,0,,0,),None


## Alternative Scraping

In [ ]:
import requests
import json
import bs4
import pandas as pd
import os
from bs4 import BeautifulSoup
import urllib.request

page = open('html_27.txt')
soup = BeautifulSoup(page, "html.parser")
soup

In [ ]:
soup.find(name='div', attrs={'class':'book-list'}).find()

## Original Code

In [ ]:
import requests
import json
import bs4
import pandas as pd
import os
from bs4 import BeautifulSoup
import urllib.request

# requeast the webpage where the content is located
URL = "http://web.archive.org/web/20040408204651/http://www.cmfu.com:80/ebook_list.asp"
page = requests.get(URL)
soup = BeautifulSoup(page.text, "html.parser")

# identify the metadata to store
link = [] ; image=[]; title = [] ; author = []; category=[]; date=[]

# retrieve all links to be scraped
for i in soup.find_all(name='tr', attrs={'bgcolor':'#ffffff'}):
  link.append('http://web.archive.org/web/20040814050024/http://www.cmfu.com/' + i.find_next('a')['href'])  #link
  ebook_id = i.find_next('a')['href'][-4:]
  image.append(f'http://web.archive.org/web/20050222132957im_/http://www.cmfu.com/bookimage/{ebook_id}.jpg') #image
  category.append(i.next.text) #category
  title.append(i.find_next('a').text) #title
  author.append(i.find_next('a').find_next(name='td',attrs={'align':'center'}).text) #author
  date.append(i.find_next('a').find_next(name='td',attrs={'align':'center'}).find_next('td').text) #date

# make a dataframe from all metadata
book_id = list(range(0, len(link)))
d = {'Id':book_id,'Title': title, 'Author':author,'Category':category,'Date':date,'Image':image,'Link':link}
df = pd.DataFrame(d)

# save the dataframe
df.to_excel('book_table.xlsx')

# create the image folder
os.mkdir('/content/images')

# download images
for i in range(len(link)):
  try:
    urllib.request.urlretrieve(image[i], f"/content/images/book{i}.jpg")
  except:
    pass

# zip all files in the folder
!zip -r /content/images.zip /content/images

# download the zip file
from google.colab import files
files.download('images.zip')
files.download('book_table.xlsx')

  adding: content/images/ (stored 0%)
  adding: content/images/book18.jpg (deflated 2%)
  adding: content/images/book3.jpg (deflated 2%)
  adding: content/images/book4.jpg (deflated 76%)
  adding: content/images/book9.jpg (deflated 2%)
  adding: content/images/book1.jpg (deflated 2%)
  adding: content/images/book29.jpg (deflated 2%)
  adding: content/images/book15.jpg (deflated 76%)
  adding: content/images/book19.jpg (deflated 76%)
  adding: content/images/book16.jpg (deflated 75%)
  adding: content/images/book22.jpg (deflated 76%)
  adding: content/images/book21.jpg (deflated 2%)
  adding: content/images/book0.jpg (deflated 2%)
  adding: content/images/book14.jpg (deflated 76%)
  adding: content/images/book2.jpg (deflated 2%)
  adding: content/images/book28.jpg (deflated 76%)
  adding: content/images/book7.jpg (deflated 2%)
  adding: content/images/book17.jpg (deflated 79%)
  adding: content/images/book8.jpg (deflated 2%)
  adding: content/images/book12.jpg (deflated 76%)
  adding: c

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>